In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
#!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sentence-transformers langchain-groq langchain_huggingface --quiet

In [3]:
print(os.environ['GROQ_API_KEY'])

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


In [4]:

from langchain.agents import tool
from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Task,Process
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [5]:
import langchain
import crewai
print(langchain.__version__)
import langchain_community
print(langchain_community.__version__)

import sys
print(sys.version)

0.1.13
0.0.29
3.12.6 (main, Sep  6 2024, 19:03:47) [Clang 15.0.0 (clang-1500.3.9.4)]


In [6]:

# Use GROQ_API_KEY
llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,

)

"""
# OR Use Ollama
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2:latest")

llm.invoke("The first man on the moon was ...")

"""


'\n# OR Use Ollama\nfrom langchain_ollama import OllamaLLM\n\nllm = OllamaLLM(model="llama3.2:latest")\n\nllm.invoke("The first man on the moon was ...")\n\n'

### Custom model and embeddings

In [7]:
# default the tool uses OpenAI for both embeddings and summarization
rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)


/Users/tharhtet/.local/share/virtualenvs/Agentic_RAGs-b0qwRdDV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OperationalError: no such column: collections.topic

In [ ]:
# Web Search Tool
web_search_tool = TavilySearchResults()

### test PDF tools

In [20]:
# test pdf tools
question  = "How did self-attention mechanism evolve in large language models?"
print(pdf_tool.run(question))

NameError: name 'pdf_tool' is not defined

In [21]:
# Test Web search Tools
question  = "How did self-attention mechanism evolve in large language models?"
web_search_tool.run(question)

NameError: name 'web_search_tool' is not defined

In [22]:
# Define Router Tool
@tool
def simple_router_tool(question: str) -> str:
    """Routes queries to the appropriate resource."""
    if "pdf" in question.lower():
        return "pdf"
    else:
        return "web"


In [23]:
question  = "How did self-attention mechanism evolve in large language models in pdf?"
simple_router_tool.run(question)


'pdf'

## Agents Creation

In [24]:
# Router Agent
router_agent = Agent(
    role="Router",
    goal="Determine if the query should go to the PDF or Web search.",
    backstory=(
        "You analyze user queries to decide whether to search a document "
        "or perform a general web search."
    ),
    llm=llm, #default ChatGPT
)


router_task = Task(
    description=(
        "Analyze the user question '{question}'. "
        "If the question involves 'document', route it to the PDF search. "
        "Otherwise, route it to web search."
    ),
    expected_output="Return 'pdf' for PDF search or 'web' for web search.",
    agent=router_agent,
    tools=[simple_router_tool],
)

2024-11-16 19:25:17,406 - 8148842560 - llm.py-llm:178 - ERROR: Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



ValidationError: 1 validation error for Task
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=StructuredTool(name='simp...er_tool at 0x3241f74c0>), input_type=StructuredTool]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type

In [25]:
rag_crew1 = Crew(
    agents=[router_agent],
    tasks=[router_task],
    verbose=True,

)

test_input = {"question": "What does the  say about machine learning?"}
result = rag_crew1.kickoff(inputs=test_input)

NameError: name 'router_task' is not defined

In [26]:
print(result)

NameError: name 'result' is not defined

In [27]:
# Retriever Agent
retriever_agent = Agent(
    role="Retriever",
    goal="Retrieve information from either a PDF or the web.",
    backstory=(
        "You are responsible for retrieving information from the appropriate "
        "resource (PDF or web) based on the router's decision."
    ),
    llm=llm,
)

retriever_task = Task(
    description=(
        "Based on the routing decision ('pdf' or 'web'), use the relevant tool to retrieve information. "
        "If 'pdf', use the PDF search tool. If 'web', use the web search tool."
    ),
    expected_output="Provide a clear, concise answer using the retrieved information.",
    context=[router_task],
    tools=[pdf_tool, web_search_tool],
    agent=retriever_agent,
)

2024-11-16 19:25:18,004 - 8148842560 - llm.py-llm:178 - ERROR: Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



NameError: name 'router_task' is not defined

In [28]:
# Crew for the System
simple_rag_crew = Crew(
    agents=[router_agent, retriever_agent],
    tasks=[router_task, retriever_task],
    verbose=True,
)

NameError: name 'router_task' is not defined

In [29]:
# Example Input
inputs = {"question": "What does the say about machine learning?"}

#inputs ={"question":"How does self-attention mechanism help large language models?"}
# Run the System
response = simple_rag_crew.kickoff(inputs)
print(response)

NameError: name 'simple_rag_crew' is not defined

In [29]:
print(response)

NameError: name 'response' is not defined